# Import necessary libraries

In [2]:
import os #menejemenisasi file dan folder
# import cv2 
import numpy as np
from sklearn.utils import shuffle
from PIL import Image #library clean data
#library untuk menampilkan gambar
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#library tensorflow untuk pelatihan model 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #library untuk augmentasi gambar
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


2022-01-10 09:00:06.939905: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


# List all data in each classes

In [3]:
#Direktori data
base_dir = 'dataset'
#Direktori data daun sakit
blast_dir = os.path.join(base_dir, 'full/LeafBlast')
#Direktori data daun sehat
brownspot_dir = os.path.join(base_dir, 'full/BrownSpot')
#Direktori data daun sehat
hispa_dir = os.path.join(base_dir, 'full/Hispa')
healthy_dir = os.path.join(base_dir, 'full/Healthy')
#Mengambil semua anama file dalam masing-masing direktori
balst_fnames = os.listdir(blast_dir )
brownspot_fnames = os.listdir(brownspot_dir)
hispa_fnames = os.listdir(hispa_dir)
healthy_fnames = os.listdir(healthy_dir)

#check file
print('total data Blast:',len(os.listdir(blast_dir)))
print('total data Brownspot:',len(os.listdir(brownspot_dir)))
print('total data Hispa:',len(os.listdir(hispa_dir)))
print('total data healthy:',len(os.listdir(healthy_dir)))

# Load Image using ImageDataGenerator

In [6]:
IMG_SIZE = 200
IMAGE_SHAPE = (IMG_SIZE, IMG_SIZE)
DATA_DIR = os.path.join(base_dir, 'full')
#VALID_DATA_DIR = os.path.join(base_dir, 'test')

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, validation_split=0.2
)


train_generator = datagen.flow_from_directory(
    DATA_DIR,
    shuffle=False,
    subset='training',
    target_size=IMAGE_SHAPE,
    class_mode='categorical'
)
valid_generator = datagen.flow_from_directory(
    DATA_DIR,
    shuffle=False,
    subset='validation',
    target_size=IMAGE_SHAPE,
    class_mode='categorical'
)

print(train_generator.class_indices)

Found 1676 images belonging to 4 classes.
Found 416 images belonging to 4 classes.


# Our model for classification

In [10]:
#create model
EPOCHS = 20
BATCH_SIZE = 32

NUM_CLASSES = 4

def save_plots(train_acc, valid_acc, train_loss, valid_loss):
    """
    Function to save the loss and accuracy plots to disk.
    """
    # accuracy plots
    plt.figure(figsize=(12, 9))
    plt.plot(
        train_acc, color='green', linestyle='-', 
        label='train accuracy'
    )
    plt.plot(
        valid_acc, color='blue', linestyle='-', 
        label='validataion accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    filepath = os.path.join(base_dir, 'figures')
    plt.savefig(filepath+'/accuracy.png')
    plt.show()
    # loss plots
    plt.figure(figsize=(12, 9))
    plt.plot(
        train_loss, color='orange', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-', 
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(filepath+'/loss.png')
    plt.show()



In [11]:

def lenet(train=False):
  
    model = tf.keras.Sequential([
        layers.Conv2D(6, (5,5), activation = 'tanh', input_shape= (IMG_SIZE,IMG_SIZE,3)),
        layers.AveragePooling2D(2),
        layers.Activation('sigmoid'),
        layers.Conv2D(16,(5,5), activation= 'tanh'),
        layers.AveragePooling2D(2),
        layers.Activation('sigmoid'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(120,(5,5), activation= 'tanh'),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(84, activation= 'tanh'),
        tf.keras.layers.Dense(NUM_CLASSES, activation= 'softmax')
    ])

  #menampilkan summary/keterangan dari model
  

    model.compile(loss='categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])
    history = None
    filepath = os.path.join(base_dir, 'models')
    if train:
    
        checkpoint_path = filepath+"/training_1/cp_le_net.ckpt"
        checkpoint_dir = os.path.dirname(checkpoint_path)

        # Create a callback that saves the model's weights
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=1)


        history = model.fit(train_generator,
            steps_per_epoch=train_generator.samples // BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=valid_generator,
            callbacks=[cp_callback],
            validation_steps= valid_generator.samples // BATCH_SIZE,
            verbose=1
            )
        train_loss = history.history['loss']
        train_acc = history.history['accuracy']
        valid_loss = history.history['val_loss']
        valid_acc = history.history['val_accuracy']

        save_plots(train_acc, valid_acc, train_loss, valid_loss)
        model.save(save_format="h5",filepath=filepath+"/model_lenet.h5")
    else:
        history = tf.keras.models.load_model(filepath+"/model_lenet.h5")
    
    return history

# Training time

# Performance measurement

In [9]:

model = get_model(train=True)

if model is not None:
    # model.summary()

    # test_labels = valid_generator.labels
    loss, acc = model.evaluate(valid_generator)
    print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


# print(valid_generator.labels)

2022-01-10 09:01:01.211418: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-10 09:01:01.212164: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-10 09:01:01.242208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 09:01:01.242637: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1050 Ti computeCapability: 6.1
coreClock: 1.62GHz coreCount: 6 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 104.43GiB/s
2022-01-10 09:01:01.242663: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-10 09:01:01.275731: I tensorflow/stream_executor/platform/d

Epoch 1/20


2022-01-10 09:01:05.713005: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-10 09:01:08.867150: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


19/52 [=========>....................] - ETA: 3:34 - loss: 1.4281 - accuracy: 0.1838

KeyboardInterrupt: 